In [1]:
import sys
import os
import warnings
os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import time
import pyarrow.parquet as pq
import pyarrow as pa
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

In [3]:
LOCAL_DATA_PATH = './context_data/'
SPLIT_SEED = 42
DATA_FILE = 'competition_data_final_pqt'
TARGET_FILE = 'public_train.pqt'
SUBMISSION_FILE = 'submit.pqt'

In [4]:
id_to_submit = pq.read_table(f'{LOCAL_DATA_PATH}/{SUBMISSION_FILE}').to_pandas()

In [5]:
# pqf = pq.ParquetFile(f'{LOCAL_DATA_PATH}/{DATA_FILE}')
pqd = pq.ParquetDataset(f'{LOCAL_DATA_PATH}/{DATA_FILE}')
pqd.files

['./context_data//competition_data_final_pqt/part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
 './context_data//competition_data_final_pqt/part-00001-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
 './context_data//competition_data_final_pqt/part-00002-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
 './context_data//competition_data_final_pqt/part-00003-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
 './context_data//competition_data_final_pqt/part-00004-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
 './context_data//competition_data_final_pqt/part-00005-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
 './context_data//competition_data_final_pqt/part-00006-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
 './context_data//competition_data_final_pqt/part-00007-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
 './context_data//competition_data_final_pqt/part-00008-aba60f69-2b63-4cc1-95ca-

In [6]:
data_agg = None
for ifile in pqd.files :

    #data = pq.read_table(f'{LOCAL_DATA_PATH}/{DATA_FILE}/{ifile}')
    data = pq.read_table(f'{ifile}')
    #pd.DataFrame([(z.name, z.type) for z in data.schema], columns = [['field', 'type']])

    data_agg0 = data.select(['user_id', 'url_host', 'request_cnt']).\
        group_by(['user_id', 'url_host']).aggregate([('request_cnt', "sum")])
    
    data_agg1 = data.select(['user_id','date', 'part_of_day', 'request_cnt']).\
        group_by(['user_id', 'date', 'part_of_day']).aggregate([('request_cnt', "sum")])
    
    if data_agg is None :
        data_agg  = data_agg0
        data_aggB = data_agg1
        print(data.to_pandas().head())
    else :
        data_agg  = pa.concat_tables([data_agg, data_agg0],promote=False)
        data_aggB = pa.concat_tables([data_aggB,data_agg1],promote=False)
    
    del data, data_agg0, data_agg1
    
data_agg  = data_agg.to_pandas()
data_aggB = data_aggB.to_pandas()

          region_name  city_name cpe_manufacturer_name cpe_model_name  \
0  Краснодарский край  Краснодар                 Apple       iPhone 7   
1  Краснодарский край  Краснодар                 Apple       iPhone 7   
2  Краснодарский край  Краснодар                 Apple       iPhone 7   
3  Краснодарский край  Краснодар                 Apple       iPhone 7   
4  Краснодарский край  Краснодар                 Apple       iPhone 7   

                      url_host cpe_type_cd cpe_model_os_type    price  \
0                ad.adriver.ru  smartphone               iOS  20368.0   
1                    apple.com  smartphone               iOS  20368.0   
2       avatars.mds.yandex.net  smartphone               iOS  20368.0   
3  googleads.g.doubleclick.net  smartphone               iOS  20368.0   
4  googleads.g.doubleclick.net  smartphone               iOS  20368.0   

         date part_of_day  request_cnt  user_id  
0  2022-06-15     morning            1    45098  
1  2022-06-19     morn

Регион   
, населенный пункт   
, производиель устройства   
, модель устроства   
, домен, с которого пришел рекламный запрос   
, тип устройства (смартфон или что-то другое)   
, операционка на устройстве   
, оценка цены устройства   
, дата   
, время дня (утро, вечер ...)   
, число запросов   
, id пользователя   

In [7]:
data_agg.head(), data_aggB.head(), data_agg.describe(), data_aggB.describe()

(   request_cnt_sum  user_id                     url_host
 0               12    45098                ad.adriver.ru
 1              491    45098                    apple.com
 2              174    45098       avatars.mds.yandex.net
 3              305    45098  googleads.g.doubleclick.net
 4              446    45098                  i.ytimg.com,
    request_cnt_sum  user_id        date part_of_day  \
 0                1    45098  2022-06-15     morning   
 1                1    45098  2022-06-19     morning   
 2                1    45098  2022-06-12         day   
 3                1    45098  2022-05-16         day   
 4                1    45098  2022-05-30         day   
 
                       url_host  
 0                ad.adriver.ru  
 1                    apple.com  
 2       avatars.mds.yandex.net  
 3  googleads.g.doubleclick.net  
 4  googleads.g.doubleclick.net  ,
        request_cnt_sum       user_id
 count     3.227767e+07  3.227767e+07
 mean      1.724852e+01  2.07442

In [82]:
#data_aggB['DW'] = pd.to_datetime(data_aggB.date).dt.day_of_week.astype(str)+'.'+data_aggB.part_of_day
#data_aggB['DW'] = data_aggB.part_of_day
data_aggB['DW'] = pd.to_datetime(data_aggB.date).dt.day_of_week.astype(str)+'.'+data_aggB.part_of_day

In [92]:
if 0 :
    data_aggC = pd.pivot_table(data_aggB[['user_id','DW','request_cnt_sum']],
                                index='user_id', columns='DW', values='request_cnt_sum', 
                                aggfunc=np.sum, fill_value=0).reset_index()
if 0 :
    data_aggC = data_aggB[['user_id','part_of_day','request_cnt_sum']].groupby(['user_id','part_of_day']).mean()
if 0 :
    data_aggC = pd.pivot_table(data_aggB[['user_id','part_of_day','request_cnt_sum']],
                                index='user_id', columns='part_of_day', values='request_cnt_sum', 
                                aggfunc=np.mean, fill_value=0).reset_index()
if 1 :
    data_aggC = pd.pivot_table(data_aggB[['user_id','DW','request_cnt_sum']],
                                index='user_id', columns='DW', values='request_cnt_sum', 
                                aggfunc=np.max, fill_value=0).reset_index()


In [93]:
data_aggC

DW,user_id,0.day,0.evening,0.morning,0.night,1.day,1.evening,1.morning,1.night,2.day,...,4.morning,4.night,5.day,5.evening,5.morning,5.night,6.day,6.evening,6.morning,6.night
0,0,1,0,3,0,74,8,0,0,11,...,3,0,2,15,3,1,8,8,4,0
1,1,35,32,35,2,24,33,18,5,32,...,36,8,21,22,18,1,24,17,18,4
2,2,12,8,2,1,7,9,2,2,19,...,3,0,16,5,13,0,19,7,11,1
3,3,6,24,7,0,13,6,1,0,7,...,9,2,12,0,6,0,19,5,7,0
4,4,19,19,37,0,17,6,25,1,35,...,54,1,20,59,36,2,32,11,18,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415312,415312,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
415313,415313,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
415314,415314,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
415315,415315,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [94]:
if 0 :
    data_aggC.columns=('user_id','a0','a1','a2','a3','a4','a5','a6')
    data_aggC['A']=data_aggC.a0+data_aggC.a1+data_aggC.a2+data_aggC.a3+data_aggC.a4+data_aggC.a5+data_aggC.a6
    data_aggC['A0']=data_aggC.a0/data_aggC.A
    data_aggC['A1']=data_aggC.a1/data_aggC.A
    data_aggC['A2']=data_aggC.a2/data_aggC.A
    data_aggC['A3']=data_aggC.a3/data_aggC.A
    data_aggC['A4']=data_aggC.a4/data_aggC.A
    data_aggC['A5']=data_aggC.a5/data_aggC.A
    data_aggC['A6']=data_aggC.a6/data_aggC.A
    data_aggC      =data_aggC.drop(['a0','a1','a2','a3','a4','a5','a6','A','A1'], axis=1)
if 0 :
    data_aggC['A']=data_aggC.day+data_aggC.evening+data_aggC.morning+data_aggC.night
    data_aggC['A000']=data_aggC.day/data_aggC.A
    data_aggC['A001']=data_aggC.evening/data_aggC.A
    data_aggC['A002']=data_aggC.morning/data_aggC.A
    data_aggC['A003']=data_aggC.night/data_aggC.A
    data_aggC = data_aggC.drop(['day','morning','evening'],axis=1)
    
data_aggC

DW,user_id,0.day,0.evening,0.morning,0.night,1.day,1.evening,1.morning,1.night,2.day,...,4.morning,4.night,5.day,5.evening,5.morning,5.night,6.day,6.evening,6.morning,6.night
0,0,1,0,3,0,74,8,0,0,11,...,3,0,2,15,3,1,8,8,4,0
1,1,35,32,35,2,24,33,18,5,32,...,36,8,21,22,18,1,24,17,18,4
2,2,12,8,2,1,7,9,2,2,19,...,3,0,16,5,13,0,19,7,11,1
3,3,6,24,7,0,13,6,1,0,7,...,9,2,12,0,6,0,19,5,7,0
4,4,19,19,37,0,17,6,25,1,35,...,54,1,20,59,36,2,32,11,18,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415312,415312,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
415313,415313,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
415314,415314,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
415315,415315,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [10]:
targets = pq.read_table(f'{LOCAL_DATA_PATH}/{TARGET_FILE}')
pd.DataFrame([(z.name, z.type) for z in targets.schema], columns = [['field', 'type']])

,field,type
0,age,double
1,is_male,string
2,user_id,int64
3,__index_level_0__,int64


In [11]:
targets.to_pandas().describe(), id_to_submit.describe()

(                 age        user_id
 count  269999.000000  270000.000000
 mean       38.789773  207547.794719
 std        11.746168  119901.875310
 min         0.000000       0.000000
 25%        30.000000  103651.500000
 50%        37.000000  207456.500000
 75%        47.000000  311565.250000
 max        91.000000  415313.000000,
              user_id
 count  144724.000000
 mean   207821.412489
 std    119885.755418
 min         6.000000
 25%    104091.500000
 50%    207961.500000
 75%    311353.250000
 max    415316.000000)

In [12]:
url_set = set(data_agg['url_host'])
print(f'{len(url_set)} urls')
url_dict = {url: idurl for url, idurl in zip(url_set, range(len(url_set)))}
usr_set = set(data_agg['user_id'])
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

199683 urls
415317 users


In [13]:
import faiss

In [14]:
%%time
values = np.array(data_agg['request_cnt_sum'])
rows = np.array(data_agg['user_id'].map(usr_dict))
cols = np.array(data_agg['url_host'].map(url_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 30, use_gpu = False, \
       calculate_training_loss = False, regularization = 0.1)

CPU times: user 3.79 s, sys: 161 ms, total: 3.95 s
Wall time: 3.99 s


In [15]:
als.model.__dict__

{'item_factors': None,
 'user_factors': None,
 '_user_norms': None,
 '_item_norms': None,
 'num_threads': 0,
 'factors': 50,
 'regularization': 0.1,
 'alpha': 1.0,
 'dtype': dtype('float32'),
 'use_native': True,
 'use_cg': True,
 'iterations': 30,
 'calculate_training_loss': False,
 'fit_callback': None,
 'cg_steps': 3,
 'random_state': None,
 '_YtY': None,
 '_XtX': None}

In [16]:
%%time
als.fit(mat)
u_factors = als.model.user_factors 
d_factors = als.model.item_factors

  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 25min 39s, sys: 2.73 s, total: 25min 41s
Wall time: 3min 17s


In [17]:
u_factors.shape, d_factors.shape, 270000+144277

((415317, 50), (199683, 50), 414277)

## Получим оценку по полу

In [97]:
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
usr_emb = pd.DataFrame(u_factors)
usr_emb['user_id'] = usr_emb.index.map(inv_usr_map)
usr_targets = targets.to_pandas()
df = usr_targets.merge(usr_emb,   how = 'inner', on = ['user_id'])
#df = usr_targets.merge(data_aggC, how = 'inner', on = ['user_id'])
df = df[df['is_male'] != 'NA']
df = df.dropna()
df['is_male'] = df['is_male'].map(int)
df['is_male'].value_counts()

CPU times: user 912 ms, sys: 80.1 ms, total: 992 ms
Wall time: 985 ms


1    135331
0    128994
Name: is_male, dtype: int64

In [98]:
%%time
x_train, x_test, y_train, y_test = train_test_split(\
    df.drop(['user_id', 'age', 'is_male'], axis = 1), df['is_male'], test_size = 0.33, random_state = SPLIT_SEED)
clf = CatBoostClassifier()
clf.fit(x_train, y_train, verbose = False)
print(f'GINI по полу {2 * m.roc_auc_score(y_test, clf.predict_proba(x_test)[:,1]) - 1:2.3f}')

GINI по полу 0.656
CPU times: user 3min 13s, sys: 10.1 s, total: 3min 23s
Wall time: 26.6 s


In [115]:
clf.fit(df.drop(['user_id', 'age', 'is_male'], axis = 1), df['is_male'], verbose = False)
#id_to_submit['is_male'] = clf.predict_proba(id_to_submit.merge(usr_emb, how = 'inner', on = ['user_id']))[:,1]

In [120]:
df.drop([ 'age', 'is_male'], axis = 1), id_to_submit, data_aggC

(        user_id         0         1         2         3         4         5  \
 0        350459 -0.636667 -0.686604  1.099947 -0.054375 -0.775412  0.629346   
 1        188276 -0.084592  0.031873  1.647439  0.767832 -0.928785  0.113753   
 2         99002 -0.787028 -0.292075  0.624598 -0.472975 -0.310680  0.526589   
 3        155506  0.106333 -0.222281  1.114539  0.226630 -0.412691 -0.353304   
 4        213873 -0.062552 -0.004976  0.216324  0.106264 -0.092225 -0.039172   
 ...         ...       ...       ...       ...       ...       ...       ...   
 264320   225374 -0.299951 -1.423215  0.053221  1.382652  0.825198  0.777674   
 264321    25776 -0.071679 -0.405494  1.069234  0.615415 -0.728076  0.107103   
 264322   148131 -0.091669 -0.201775  1.378421 -0.119591 -0.506074 -0.090850   
 264323   205570 -0.901080 -0.383949  0.754910  0.408689 -0.458831  0.340509   
 264324   103148 -1.110192 -1.034697  0.255509  1.657812 -0.534860  1.627752   
 
                6         7         8 

In [116]:
id_to_submit['is_male'] = clf.predict_proba(id_to_submit.merge(df.drop(['age', 'is_male'], axis = 1), how = 'inner', on = ['user_id']))[:,1]

No objects info loaded


ValueError: Length of values (0) does not match length of index (144724)

## Получим оценку по возрасту

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

%matplotlib inline
sns.set_style('darkgrid')

In [ ]:
def age_bucket(x):
    return bisect.bisect_left([18,25,35,45,55,65], x)

In [ ]:
df = usr_targets.merge(usr_emb, how = 'inner', on = ['user_id'])
df = df[df['age'] != 'NA']
df = df.dropna()
df['age'] = df['age'].map(age_bucket)
sns.histplot(df['age'], bins = 7)

In [127]:
x_train, x_test, y_train, y_test = train_test_split(\
    df.drop(['user_id', 'age', 'is_male'], axis = 1), df['age'], test_size = 0.33, random_state = SPLIT_SEED)

clf = CatBoostClassifier()
clf.fit(x_train, y_train, verbose = False)
print(m.classification_report(y_test, clf.predict(x_test), \
                            target_names = ['<18', '18-25','25-34', '35-44', '45-54', '55-65', '65+']))

CatBoostError: catboost/cuda/cuda_lib/cuda_base.h:281: CUDA error 35: CUDA driver version is insufficient for CUDA runtime version

x_train, x_test, y_train, y_test = train_test_split(\
    df.drop(['user_id', 'age', 'is_male'], axis = 1), df['age'], test_size = 0.33, random_state = SPLIT_SEED)

clf = CatBoostClassifier()
clf.fit(x_train, y_train, verbose = False)
print(m.classification_report(y_test, clf.predict(x_test), \
                            target_names = ['<18', '18-25','25-34', '35-44', '45-54', '55-65', '65+']))

In [105]:
clf.fit(df.drop(['user_id', 'age', 'is_male'], axis = 1), df['age'], verbose = False)
id_to_submit['age'] = clf.predict(id_to_submit[['user_id']].merge(usr_emb, how = 'inner', on = ['user_id']))

## Сабмит

In [59]:
id_to_submit.head()

,user_id,is_male,age
221301,221301,0.976778,2
31271,31271,0.558827,3
211594,211594,0.493528,2
253119,253119,0.532796,2
192578,192578,0.866905,2


In [60]:
id_to_submit.to_csv(f'{LOCAL_DATA_PATH}/submission3.csv', index = False)

In [61]:
! head $LOCAL_DATA_PATH/submission1.csv

user_id,is_male,age
221301,0.9523721503994212,2
31271,0.540058202285835,3
211594,0.4921148361193288,2
253119,0.5031936669434278,2
192578,0.7343094598406392,2
268458,0.28223318330597313,2
205507,0.5256998725263032,3
341343,0.6329608219845485,3
282144,0.026135216529779966,2


# Скор на лидерборде

In [32]:
context_scorer(submission, answers)

NameError: name 'context_scorer' is not defined